In [1]:
import spacy
nlp = spacy.load('en_core_web_md')

In [2]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

In [3]:
parts = []
corp = ''
with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/text/ep-09-03-11-018-en.txt') as f:
    for index, line in enumerate(f):
        corp = corp + line
        if index % 20 == 0:
            parts.append(corp)
            corp = ''

In [4]:
docs = []
for part in parts:
    doc = nlp(part)
    docs.append(doc)


In [5]:
sweCorp = ''
with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/text/ep-09-03-11-018-sv.txt') as f:
    for line in f:
        sweCorp = sweCorp + line


In [6]:
tokenizerEng = spacy.load('en_core_web_md', disable=['tagger', 'parser', 'ner'])
tokenizerSwe = spacy.load('../Models/SwedishModel', disable=['tagger', 'parser', 'ner'])

In [7]:
startTokenEng = 0
startTokenSwe = 0
alignments = {}

with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/text/ep-09-03-11-018-en.txt') as engDoc:
    with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/alignments/ali.sym') as fa:
        with open('../Data/Datasets/Europarl/Documents/ep-09-03-11-018/text/ep-09-03-11-018-sv.txt') as sweDoc:
            for lineAli in fa:
                lineEngDoc = engDoc.readline()
                lineSweDoc = sweDoc.readline()
                tokEng = tokenizerEng(lineEngDoc)
                tokSwe = tokenizerSwe(lineSweDoc)

                alignmentListTemp = lineAli.split()
                for ali in alignmentListTemp:
                    aliPair = ali.split('-')
                    key = str(int(aliPair[0]) + startTokenEng)
                    if alignments.get(key) is None:
                        alignments[key] = [int(aliPair[1]) + startTokenSwe]
                    else:
                        alignments.get(key).append(int(aliPair[1]) + startTokenSwe)

                startTokenEng = startTokenEng + len(tokEng)
                startTokenSwe = startTokenSwe + len(tokSwe)


In [9]:
spansList = []

for index, cluster in enumerate(doc._.coref_clusters):
    clusterSpansList = []
    for mention in doc._.coref_clusters[index].mentions:
        mentionSpan = []
        aligns = []
        for i in range(mention.start, mention.end):
            key = str(i)
            if alignments.get(key) is not None:
                for wordIndex in alignments.get(key):
                    aligns.append(wordIndex)
            else:
                continue
        aligns = sorted(set(aligns))
        maxContinuousSpan = []
        if len(aligns) == 0:
            maxContinuousSpan.append(-1)
        elif len(aligns) == 1:
            maxContinuousSpan.append(aligns[0])
            maxContinuousSpan.append(aligns[0] + 1)
        else:
            start = aligns[0]
            end = aligns[0]
            spansInAligns = []
            current = 0

            for i, tokenIndex in enumerate(aligns):
                if tokenIndex == start + current:
                    end = tokenIndex
                    current += 1
                    if tokenIndex == aligns[-1]:
                        currentSpan = [start, end + 1]
                        spansInAligns.append(currentSpan)
                    else:
                        continue
                else:
                    currentSpan = [start, end + 1]
                    spansInAligns.append(currentSpan)
                    start = tokenIndex
                    end = tokenIndex
                    current = 1
            #maxContinuousSpan.append(spansInAligns[0])
            maxlength = 0
            for conSpan in spansInAligns:
                if conSpan[1] - conSpan[0] > maxlength:
                    maxlength = conSpan[1] - conSpan[0]
                    maxContinuousSpan = conSpan
                else:
                    continue

        clusterSpansList.append(maxContinuousSpan)
    spansList.append(clusterSpansList)

In [10]:
import pickle

with open('../Coref/pickleObjects/corefSpansObject', 'wb') as f:
    pickle.dump(spansList, f)
